In [1]:
import pandas as pd
import numpy as np 
import shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnet_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

Using TensorFlow backend.


In [5]:
import shutil
import os
import tensorflow as tf

class TFRecordExtractor:
	def __init__(self, tfrecord_file):
		self.tfrecord_file = os.path.abspath(tfrecord_file)
	
	# def extract_fn(self, data_record):
    	# features = {
     #    	# Extract features using the keys set during creation
     #    	'int_list1': tf.FixedLenFeature([], tf.int64),
     #    	'float_list1': tf.FixedLenFeature([], tf.float32),
     #    	'str_list1': tf.FixedLenFeature([], tf.string),
     #    	# If size is different of different records, use VarLenFeature 
     #    	'float_list2': tf.VarLenFeature(tf.float32)
    	# }
    	# sample = tf.parse_single_example(, features)
    	# return sample
	
	
	def extract_fn(self, data_record):
		features = {
			'int_list1': tf.FixedLenFeature([], tf.int64),
			'float_list1': tf.FixedLenFeature([], tf.float32),
			'str_list1': tf.FixedLenFeature([], tf.string),
			'float_list2': tf.VarLenFeature(tf.float32)
		}
		sample = tf.parse_single_example(data_record, features)
		return sample
		
	# def extract_fn(self, tfrecord):
	# 	# Extract features using the keys
	# 	features = {
     #        'train/image': tf.FixedLenFeature([], tf.string),
     #        'train/label': tf.FixedLenFeature([], tf.int64)
     #    }
	# 	
	# 	# Extract the data record
	# 	sample = tf.parse_single_example(tfrecord, features)
	# 	image = tf.image.decode_image(sample['train/image'])
	# 	img_shape = (64, 64, 3)
	# 	label = sample['train/label']
	# 	return [image, label, img_shape]
	
	
	def extract_image(self):
		folder_path = './ExtractedImages'
		shutil.rmtree(folder_path, ignore_errors=True)
		os.mkdir(folder_path)
		
		# Pipeline
		dataset = tf.data.TFRecordDataset([self.tfrecord_file])
		dataset = dataset.map(self.extract_fn)
		iterator = dataset.make_one_shot_iterator()
		next_image = iterator.get_next()
		
		with tf.Session() as sess:
			sess.run(tf.global_variables_initializer())
			try:
				# Keep extracting until tfrecord's end
				while True:
					image_data = sess.run(next_image)
					
					# compare image shape, should be the same
					if not np.array_equal(image_data[0].shape, (64, 64, 3)):
						print('Image {} not decoded properly'.format(image_data[2]))
						continue
					
					save_path = os.path.abspath(os.path.join(folder_path, image_data[1].decode('utf-8')))
					mpimg.imsave(save_path, image_data[0])
					print('Save path = ', save_path, ', label = ', image_data[2])
			except:
				pass

		
data_path_train = os.path.abspath('./TFRecord/kanji_train.tfrecord')
train_nimg = 0
for record in tf.python_io.tf_record_iterator(data_path_train):
		train_nimg += 1
data_path_test = os.path.abspath('./TFRecord/kanji_test.tfrecord')
test_nimg = 0
for record in tf.python_io.tf_record_iterator(data_path_test):
		test_nimg += 1
data_path_val = os.path.abspath('./TFRecord/kanji_val.tfrecord')
val_nimg = 0
for record in tf.python_io.tf_record_iterator(data_path_test):
			val_nimg += 1

# Initialize all tfrecord paths

print("Amount of training images: " + str(train_nimg))
print("Amount of testing images: " + str(test_nimg))
print("Amount of validation images: " + str(val_nimg))

train_dataset = tf.data.TFRecordDataset(data_path_train)
test_dataset = tf.data.TFRecordDataset(data_path_test)
val_dataset = tf.data.TFRecordDataset(data_path_val)

train = TFRecordExtractor(data_path_train)
train.extract_image()
test = TFRecordExtractor(data_path_test)
test.extract_image()
val = TFRecordExtractor(data_path_val)
val.extract_image()



Amount of training images: 29754
Amount of testing images: 6376
Amount of validation images: 6376
